In [ ]:
####### ESTE CODIGO ES PARA QUE ME DEVUELVA UN ARCHIVO LLAMADO RESUMEN, EN EL CUAL VOY A TENER EL LISTADO DE LOS VIDEOS, LA CONDICION DEL INDIVIDUO, Y PARA CADA VIDEO EL PROMEDIO DE LAS VARIABLES #####################3

import os
import pandas as pd

# Definir la carpeta de entrada y el archivo de resumen de salida
input_folder = r'C:\Users\Dell\Desktop\Prueba11'
output_file = r'C:\Users\Dell\Desktop\Prueba11\Resumen.xlsx'

# Listar los archivos en la carpeta de entrada
excel_files = [f for f in os.listdir(input_folder) if f.endswith('.xlsx')]

# Definir los encabezados y las variables a buscar
variables = ['eye_mean_x', 'eye_mean_y', 'tail_mean_x', 'tail_mean_y', 
             'head_velocity', 'tail_velocity', 'angular_velocity', 'sum_tail_angles', 
             'tail6_velocity', 'head_tail_distance', 'head_acceleration', 
             'tail_acceleration', 'angular_acceleration', 'head_jerk', 
             'tail_jerk', 'sum_curvature', 'angle_velocity_heading', 
             'curvature_rate']

# Crear un DataFrame para almacenar los resultados
resumen_df = pd.DataFrame(columns=["Nombre del video", "Condicion"] + variables)

# Función para obtener la condición del nombre del archivo
def obtener_condicion(filename):
    partes = filename.split('_')
    if 'S' in partes:
        return 'S'
    elif 'A' in partes:
        return 'A'
    else:
        return 'Desconocida'

# Procesar cada archivo
for filename in excel_files:
    file_path = os.path.join(input_folder, filename)
    try:
        # Obtener la condición del archivo
        condition = obtener_condicion(filename)
        
        # Cargar la hoja "Promedios" (hoja 2)
        xls = pd.ExcelFile(file_path)
        df = pd.read_excel(xls, sheet_name=1)  # Hoja 2 (índice 1)
        
        # Verificar si las dos primeras columnas tienen los encabezados "Variable" y "Valor"/"Promedio"
        if df.shape[1] >= 2:
            variable_col = df.columns[0]
            value_col = df.columns[1]  # No importa si es "Valor" o "Promedio"
            
            # Crear un diccionario para almacenar los valores de las variables
            video_data = {'Nombre del video': filename, 'Condicion': condition}
            
            # Iterar sobre las variables y extraer los valores correspondientes
            for var in variables:
                # Comprobar si la variable está presente con o sin el prefijo 'avg_'
                var_with_prefix = f'avg_{var}'
                if var in df[variable_col].values:
                    value = df[df[variable_col] == var][value_col].values[0]
                elif var_with_prefix in df[variable_col].values:
                    value = df[df[variable_col] == var_with_prefix][value_col].values[0]
                else:
                    value = None  # Si no existe la variable, dejar como None
                
                video_data[var] = value
            
            # Agregar la fila al DataFrame de resumen
            resumen_df = pd.concat([resumen_df, pd.DataFrame([video_data])], ignore_index=True)
        else:
            print(f"Hoja 'Promedios' no tiene el formato esperado en {filename}")
    
    except Exception as e:
        print(f"Error procesando el archivo {filename}: {e}")

# Guardar el DataFrame de resumen en un nuevo archivo Excel
resumen_df.to_excel(output_file, index=False)
print(f"Resumen creado exitosamente en {output_file}")

